In [ ]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

class ArizonaPlantVectorStore:
    def __init__(self, 
                 embedding_model_name='all-MiniLM-L6-v2',
                 collection_name='arizona_plants'):
        """
        Initialize the vector store
        
        Args:
            embedding_model_name: SentenceTransformer model name
            collection_name: Name for the Qdrant collection
        """
        self.collection_name = collection_name
        
        # Initialize embedding model
        print(f"Loading embedding model: {embedding_model_name}")
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.embedding_dim = self.embedding_model.get_sentence_embedding_dimension()
        print(f"✓ Model loaded (dimension: {self.embedding_dim})")
        
        # Initialize Qdrant client
        self.client = QdrantClient()
        print("✓ Qdrant client initialized")
    
    def search(self, query: str, limit: int = 5):
        """
        Search for relevant documents using pre-computed embeddings
        
        Args:
            query: Search query string
            limit: Number of results to return
            
        Returns:
            List of search results with metadata
        """
        print(f'Searching for: "{query}"')
        
        # Encode the query using SentenceTransformer
        query_embedding = self.embedding_model.encode(query)
        
        # Search in Qdrant using the embedding vector
        search_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding.tolist(),  # Pass the vector directly, not wrapped in Document
            limit=limit,
            with_payload=True
        )

        # Format results
        results = []
        for point in search_results.points:  # Note: .points here
            result = {
                'id': point.payload.get('id', 'N/A'),
                'score': point.score,
                'title': point.payload.get('title', 'Untitled'),
                'content': point.payload.get('content', ''),
                'type': point.payload.get('type', ''),
                'source': point.payload.get('source', ''),
                'metadata': point.payload.get('metadata', {})
            }
            results.append(result)
        
        print(f"✓ Found {len(results)} results")
        return results


In [ ]:
from openai import OpenAI

class ArizonaPlantRAG:
    def __init__(self, vector_store: ArizonaPlantVectorStore):
        self.vector_store = vector_store
        self.openai_client = OpenAI() #-- Uncomment if using standard OpenAI client
    
    def build_prompt(self, query, search_results):
        """
        Build a prompt for the LLM using retrieved documents
        
        Args:
            query: User's question
            search_results: List of relevant documents from vector search
            
        Returns:
            Formatted prompt string
        """
        # Build context from search results
        context_parts = []
        
        for i, result in enumerate(search_results, 1):
            context_parts.append(f"Document {i} (Score: {result['score']:.3f}):")
            context_parts.append(f"Title: {result['title']}")
            context_parts.append(f"Source: {result['source']}")
            context_parts.append(f"Content: {result['content']}")
            context_parts.append("")  # blank line
        
        context = "\n".join(context_parts)
        
        # Build the full prompt
        prompt = f"""You are an expert on Arizona desert plants. Answer the user's question based on the provided context from authoritative sources.

        Context from relevant documents:
        {context}

        User Question: {query}

        Instructions:
        - Provide a clear, detailed answer based on the context above
        - If the context contains scientific names, include them
        - If mentioning care instructions, be specific about Arizona conditions
        - If the context doesn't fully answer the question, say so
        - Cite which document(s) you're drawing from (e.g., "According to Document 1...")

        Answer:""".strip()
        
        return prompt


    def llm(self, prompt):
        """
        Call OpenAI API to generate answer
        
        Args:
            prompt: Full prompt with context and question
            
        Returns:
            Generated answer string
        """
        
        response = self.openai_client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,  # Slightly creative but mostly factual
            max_tokens=500    # Adjust based on your needs
        )
        
        return response.choices[0].message.content


    def rag(self, query):
        """
        Complete RAG pipeline: Retrieve relevant docs and generate answer
        
        Args:
            query: User's question
            vector_store: Initialized ArizonaPlantVectorStore instance
            
        Returns:
            Generated answer string
        """
        # Step 1: Retrieve relevant documents
        print("\n" + "="*60)
        print("RAG Pipeline")
        print("="*60)
        search_results = self.vector_store.search(query, limit=5)
        
        # Step 2: Build prompt with context
        prompt = self.build_prompt(query, search_results)
        
        # Optional: Print prompt for debugging
        # print("\nPrompt being sent to LLM:")
        # print("-"*60)
        # print(prompt)
        # print("-"*60)
        
        # Step 3: Generate answer
        print("Generating answer...")
        answer = self.llm(prompt)
        
        print("✓ Answer generated")
        print("="*60 + "\n")
        
        return answer


In [ ]:
assistant = ArizonaPlantRAG(ArizonaPlantVectorStore())

In [ ]:
query = "What are some drought-tolerant plants suitable for a home garden in Phoenix, Arizona?"

answer = assistant.rag(query)
print("Question:", query)
print("\nAnswer:", answer)